<a href="https://colab.research.google.com/github/decoz/mlclass/blob/master/10_pytorch_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mnist Data 로 학습하기

MNIST database 란 (Modified National Institute of Standards and Technology database) 의 약자로서 미국에서 사람의 손글씨 학습을 위해 수집된 숫자 손글씨의 모음데이터이다. 원래는 Nist 데이터라고 불렸는데 이를 머신러닝에 적합하게 수정함으로서 Modified NIST 의 약자로 MNIST라 불린다.  주로 28x28 의 작은 60000 개의 학습이미지와 10000 개의 테스트 이미지로 구성되어있으며 각각에 대한 레이블 값도 제공된다. 

요즘은 28x28 로 구성된 작은 학습이미지 샘플 자체를 MNIST라고 부르기도 한다. 

## Mnist Data 생성

Mnist 는 Torchvision 이라는 torch 의 보조 라이브러리통해 제공된다. 

In [ ]:
import torch as tc
import torchvision as tcv


이제  Mnist 데이터를 읽어들여보겠다. 약 3-4분정도가 소요된다. 

In [ ]:
mnist = tcv.datasets.MNIST( root = './data', train = True,  download=True)

이제 읽어들인 데이터를 한번 살펴보자.  dir 은 클래스의 각종 멤버 변수와 함수를 보여준다. 

In [ ]:
print("type:",type(mnist))
dir(mnist)


보면 멤버중에 data 라는 멤버가 보인다.  이 내용을 한번 보기로 하자

In [ ]:
print("data type:", mnist.data)
print("data shape:",mnist.data.shape)


보면 data 멤버는 60000 x 28 x 28 의 구조로 저장되어 있는 텐서임을 알 수 있다. 이를 통해 한 이미지의 크기가 28x28 이며 총 60000 개의 이미지가 있음을 유추해볼 수 있다.  이번에는 train_labels 도 확인해보자. 



In [ ]:
print("train labels:", mnist.train_labels)
print("train labels shape:", mnist.train_labels.shape)

train_labels 는 6만개의 이미지가 각각 어떤 숫자를 표현하는지가 나와있다. 

## Mnist Data 시각화

Mnist 가 실제로 어떤 형태의 이미지인지 확인하기 위해 첫번째 이미지를 추출해 imshow 를 사용해 표시해보자 

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 

first = mnist.data[0]
plt.imshow(first, cmap = 'gray')

이번에는 처음 4개의 이미지를 추출해서 sample 이라는 텐서 를 생성해보도록 핵ㅆ다. 

In [ ]:
sample = mnist.data[:4]
slabels = mnist.train_labels[:4]

4개의 이미지도 한번 확인해보자. 

In [ ]:
import matplotlib.pyplot as plt
for img in sample:
  plt.imshow(img, cmap = 'gray')
  plt.show()

print(slabels)


## Sample 학습해보기 

이제 위에서 추출한 4개의 샘플을 신경망을 통해서 학습하하도록 해보겠다. linear모델을 위해 4x28x28 형태인 sample 을 4x784 의 2차원 배열로 변환해 x 를 만든다. 



In [ ]:
x = sample.view(-1, 28 * 28)
print(x.shape)
x


그런데 위의 x 는 정수형이므로 신경망의 입력으로는 적절치 안다. 이를 위해서는 다음 구문을 이용해  Float으로 변환을 해준다. 

In [ ]:
x = x.type(tc.FloatTensor)
x

### One-hot encoding 

그런데 label 값은 0,1,2,3 같은 정수형으로 되어있다. 하지만 이것은 분류문제이므로 이것을 학습할 때는 
```
0 -> [1,0,0,0 .. ]
1 -> [0,1,0,0 .. ]
2 -> [0,0,1,0 .. ]

```
이런식으로 각각의 클래스마다 출력값을 주는 방식을 사용하는게 효율적이다.  이렇게 각각의 labels 마다 해당하는 출력을 주고 해당하는 출력만 타겟을 1로 하고 나머지는 0으로 세팅하는 방법을 One-hot encoding 이라고 한다. 


torch 에서는 이런 함수를 이용해 scatter 라는 함수를 사용한다. scatter 는 특정 인덱스에 배열에 특정 값을 뿌려주는 역할을 한다. 

In [ ]:
t = tc.tensor([0,0,0,0,0])
idx = tc.tensor([1,3])

print( t.scatter(0,idx,9))

처음 0 은 배열의 차원을 의미하는 이를 통해 다음과 같이 2차원에도 적용이 가능하다. 

In [ ]:
t = tc.tensor([[0,0,0,0,0],[0,0,0,0,0]])
idx = tc.tensor([[1],[3]])
print( t.scatter(1,idx,1))

이제 이것을 이용해 위의 4개의 labels 값에 해당하는 one-hot encoding y를 만들 수 있다. 

In [ ]:
y = tc.zeros(4,10).scatter_(1,slabels.view(-1,1), 1)
y

### nn 모델 구성 

이제 앞에서 배운 nn 모델을 활용해 784 크기의 입력과 10개의 출력 10개의 히든을 갖는 모델을 구성해보도록 하겠다. 

In [ ]:
import torch.nn as nn 
model = nn.Sequential(
    nn.Linear(28 * 28, 10),
    nn.Sigmoid(),
    nn.Linear(10, 10),
    nn.Sigmoid()
)

model(x)


이제 optimizer 를 생성한다. 

In [ ]:
import torch.optim as optim

optimizer = optim.SGD( model.parameters() , lr = 0.1)



이제 학습을 시작해보도록 하겠다. 

In [ ]:
ds = []
for step in range(10000):
  o = model(x)
  d = nn.functional.mse_loss(o, y)
  optimizer.zero_grad() 
  d.backward()
  optimizer.step() 
  
  ds += [d.item()]
  if step % 100 == 0 : 
    print("err:",d.item())



이제 오류율 그래프를 확인한다. 

In [ ]:

plt.plot(ds)

마지막 출력은 텐서변수 o 를 통해 확인이 가능하다.  출력은 반올림을 통해 0과  1로 확인을 해준다.  또한 argmax 를 사용하면 직접 label 값을 추출할 수도 있다. 

In [ ]:
print(tc.round( o ))
print(tc.argmax(o, dim = 1))

## 1000개의 Mnist 데이터 학습해보기 

이번에는 조금 더 제대로된 학습을 위해서 1000개의 데이터를 학습해보도록 하겠다. 이제  위의 mnist data 에서 한번에 1000개의 데이터를 빼내보도록 하겠다. 

In [ ]:
tr_n = 1000 #트레이닝 데이터의 갯수

x = mnist.data[:tr_n].view(-1,28*28).type(tc.FloatTensor)
labels = mnist.train_labels[:tr_n]
y =  tc.zeros(tr_n,10).scatter(1, labels.view(-1,1), 1)


이제 위에 있는 모델을 통해서 학습을 수행해보도록 하겠다. 



In [ ]:
ds = []
for step in range(1000):
  o = model(x)
  d = nn.functional.mse_loss(o, y)
  optimizer.zero_grad() 
  d.backward()
  optimizer.step() 

  ds += [d.item()]
  if step % 100 == 0 : 
    print("err:",d.item())

plt.plot(ds)

### 매칭 결과 체크

학습이 잘 수행되는 듯 할 것이다. 하지만 과연 1000개의 이미지중에 몇개나 맞추고 있는걸까? 이를 확인하는 방법을 알아보자. 

In [ ]:
print(tc.argmax(o, dim = 1))

위의 출력은 실제 출력값중에 최대값을 출력해본 것이다. 이것을 labels 와 비교하면 다음과 같다. 

In [ ]:
print( tc.argmax(o,dim=1) == labels)

위의 결과중에 True 는 실제 학습한 신경망이 올바른 레이블에 해당하는 노드를 최대값으로 출력했음을 의미한다.  그러므로 True 의 숫자를 카운트하고 이를 1000으로 나누면 실제 매칭률을 구할 수 있다. True 숫자는 tc.sum() 으로 구할 수 있다. 

In [ ]:
match_rate = tc.sum( tc.argmax(o, dim =1) == labels ) / 1000
print(match_rate)

이제 매칭률을 출력하며 학습을 해보도록 하겠다. 


In [ ]:
ds = []
for step in range(1000):
  o = model(x)
  d = nn.functional.mse_loss(o, y)
  optimizer.zero_grad() 
  d.backward()
  optimizer.step() 

  ds += [d.item()]
  if step % 100 == 0 : 
    match_rate = tc.sum( tc.argmax(o, dim =1) == labels ) / tr_n

    print("match rate:{:.3f}, err:{:.5f}".format(match_rate,d.item()) ) 

plt.plot(ds)

### 더 복잡한 모델 사용하기

아마 매칭률이 0.8정도에서 정체된걸 볼 수 있을것이다. 그러면 이게 한계일까?  사실 약 800개의 입력을 처리하기에는 히든노드 10개는 너무나 부족하다. 이번에는 높은 매칭률을 가질 수 있는 모델을 구성해보자. 



In [ ]:
import torch.nn as nn 
import torch.optim as optim

model = nn.Sequential(
    nn.Linear(28 * 28, 800),
    nn.Sigmoid(),
    nn.Linear(800, 80),
    nn.Sigmoid(),    
    nn.Linear(80, 10),
    nn.Sigmoid()
)



optimizer = optim.SGD( model.parameters() , lr = 0.1)
model(x)

위의 모델은 히든을 800 - 80 으로 2단계로 배치했다.  이 모델은 95% 이상의 매칭률을 보일 수 있는 구성이다. 하지만  위의 노드를 1000회만 돌려봐도 속도가 매우 느리다는 것을 알 수 있을 것이다. 사실 중간노드 800개는 784 x 800 의 웨이트를 생성하기 때문에 실제로 80배 이상 계산이 늘어난다. 

In [ ]:
ds = []
for step in range(1000):
  o = model(x)
  d = nn.functional.mse_loss(o, y)
  optimizer.zero_grad() 
  d.backward()
  optimizer.step() 

  ds += [d.item()]
  if step % 100 == 0 : 
    match_rate = tc.sum( tc.argmax(o, dim =1) == labels ) / 1000
    print("match rate:{:.3f}, err:{:.5f}".format(match_rate,d.item()) ) 


### GPU를 사용한 학습

이 문제를 해결하는 가장  좋은 방법은 gpu 사용하는 것이다. 지금까지 우리는 cpu에서 계산을 수행했다. 하지만 gpu 를 사용하면 10배 이상의 속도를 얻을 수 있다.  gpu를 사용하기 위해서는 먼저 다음 구문을 이용해 'cuda' device를 생성한다. 

```
cuda = tc.device('cuda')
```

그 후에 계산에 사용되는 모델과 텐서를 다음과 같이 gpu로 업로드한다. 

```
x = x.to(cuda)
y = y.to(cuda)
labels = labels.to(cuda)
model.to(cuda)
```

model 은 따로 대입문을 사용하지 안아도 무방하다.  그러면 이제 위의 구문을 실제로 수행해보겠다. 


In [ ]:
cuda = tc.device('cuda')
x = x.to(cuda)
y = y.to(cuda)
labels = labels.to(cuda)
model.to(cuda)

이제 위의 코드를 다시 실행핸다. 

In [ ]:
ds = []
for step in range(10000):
  o = model(x)
  d = nn.functional.mse_loss(o, y)
  optimizer.zero_grad() 
  d.backward()
  optimizer.step() 

  ds += [d.item()]
  if step % 100 == 0 : 
    match_rate = tc.sum( tc.argmax(o, dim =1) == labels ) / tr_n
    print("match rate:{:.3f}, err:{:.5f}".format(match_rate,d.item()) ) 

이제는 매우 빠른 속도로 매칭률 0.99 이상을 이룰 수 있을것이다. 

### 테스트 셋 사용해보기 

그런데 과연 트레이닝 데이터셋에 최적화된 모델이 정말로 숫자를 판별할 수 있을까? 그냥 저 데이터만 달달 외운것에 지나지 안을까?  실제로 특징의 학습을 떠나 학습데이터에만 지나치게 최적화될 경우 오히려 일반적인 판별능력이 떨어지는 현상이 존재한다.  이를 '과적합' 이라고 한다. 

이를 확인하기 위해서는 학습셋에 해당하지 안는 검증용 이미지를 활용해 검증을 하는 방법이 있다.  현재 0~999번 데이터를 학습에 사용하므로 1000~1099 번까지 100개의 이미지를 통해 검증을 해보도록 하겠다. 

먼저 테스트 셋 이미지와 레이블을 만든다. 

In [ ]:
tt_n = 100
tx = data.data[tr_n:tr_n+tt_n].view(-1,28*28).type(tc.FloatTensor).to(cuda)
ty = data.train_labels[tr_n:tr_n+tt_n].to(cuda)


이제 이를 model 을 통해 검증을 해보도록 하겟다. 단 현재 모델은 gpu 에 올라가 있기 때문에 위의 tx,ty 도 gpu에 올려야 한다. 

In [ ]:
tx = tx.to(cuda)
ty = ty.to(cuda)

이제 현재 모델로 테스트용 아웃풋을 만든 후에 매칭률을 보도록 하겠다.  model(tx)를 통해서 test 이미지의 예측값을 구한 후에 위에서와 마찬가지로 argmax 로 label 값을 추출하고  이를 ty 와 비교한 후에 true를 카운트한다. 

In [ ]:
test_match = tc.sum( tc.argmax( model(tx) , dim = 1) == ty ) / 100 
print(test_match)

아마 약 80~90% 의 매칭률을 보였을 것이다.  그러면 이제 test_match의 변화도 보면서 위의 과정을 다시 수행해보도록 한다.  모델을 다시 생성해도 되지만 매번 수백만개의 웨이트를 다시  생성해서 gpu 에 올리는 시간을 절약하기 위해 다음 함수를 사용해보도록 하겠다. 

In [ ]:
def weight_reset(model):
  for name,m in model.named_children():
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):        
        print("reset:", name)
        m.reset_parameters()

weight_reset(model)

이제 새로 실행하며 match rate 와 test match 의 변화를 구경해보자

In [ ]:

ds = []
for step in range(10000):
  o = model(x)
  d = nn.functional.mse_loss(o, y)
  optimizer.zero_grad() 
  d.backward()
  optimizer.step() 

  ds += [d.item()]
  if step % 1000 == 0 : 
    match_rate = tc.sum( tc.argmax(o, dim =1) == labels ) / tr_n
    test_match = tc.sum( tc.argmax( model(tx) , dim = 1) == ty ) / tt_n 
    print("match rate:{:.3f}, test match:{:.3f} err:{:.5f}".format(match_rate, test_match, d.item()) ) 

### <font color = '#FF7050'> 실습 : 10000 개로 학습하기 
위에서 한 과정을 트레이닝셋 10000개 테스트셋 1000 개로 다시 학습하고  그 결과를 확인해보자. 
</font>

In [ ]:
## 실습에 필요한 트레이닝, 테스트 데이터를 만드세요



In [ ]:
## 위의 데이터를 학습하며 학습결과를 확인하세요


### CrossEntropy 사용해보기 

사실 위에서 우리는 어디까지나 Simgoid 값을 기준으로 mse 에러를 계산했다. 하지만 one-hot encoding 에 더 최적화된 loss 함수가 존재하는데 이는 타겟 출력값에 대한 평중을 더 가능하게 해준다.  

이 함수는 nn.CrossEntropyLoss() 를 사용하는데 특이하게 이 함수는 매우 다양한 옵션에 의해 그 구조자체가 바귀는 경우가 있기 때문에 함수 CrossEntropyLoss() 는 바로 값을 입력받는게 아니라 그 계산을 위한 함수를 생성해준다. 그러므로 다음 구문으로  먼저 함수를 생성한다. 

In [ ]:
Loss = nn.CrossEntropyLoss()

또한 CrossEntropyLoss 는 자체적으로 One-hot encoding 을 내장하고 있으므로 인코딩된 y값을 바로 사용한다.  그러므로 d 는 다음과 같이 변형된다. 

```
d = nn.functional.mse_loss(o, y)
->
d = Loss(o, labels) 
```

이제 이걸 사용해 학습을 수행한 후에 수렴속도를 비교해보자. 


In [ ]:
#weight_reset(model)

ds = []
for step in range(10000):
  o = model(x)
 
  d = Loss(o, labels)
  optimizer.zero_grad() 
  d.backward()
  optimizer.step() 
 
  ds += [d.item()]
  if step % 1000 == 0 : 
    match_rate = tc.sum( tc.argmax(o, dim =1) == labels ) / 1000
    test_match = tc.sum( tc.argmax( model(tx) , dim = 1) == ty ) / 100 
    print("match rate:{:.3f}, test match:{:.3f} err:{:.5f}".format(match_rate, test_match, d.item()) ) 
    

<br><br>


## 커스텀 이미지 판별

이번에는 우리가 직접 그림판에 이미지를 만들어서 처리해보도록 하겠다. 먼저 그림판에서 숫자 이미지를 만든 후에 이를 업로드해보자. 

<img src = "https://github.com/decoz/mlclass/blob/master/images/4.png?raw=true" align = "left">




### 단일 이미지 판별

이제 이 이미지를 로딩해서 텐서로 변환해보겠다. 

In [ ]:
from PIL import Image

im = Image.open("4.png").convert('L')
im = im.resize((28,28))
x = tc.tensor( im.getdata() )
plt.imshow(x.reshape(28,28), cmap='gray')


위의 이미지는 배경이 흰색이므로 검은색으로 변환한다. 

In [ ]:
x = 255 - x 
plt.imshow(x.reshape(28,28), cmap='gray')



이제 이 이미지를 model 에 넣어보도록  하겠다. 

In [ ]:
x = x.type(tc.FloatTensor).to(cuda)
tc.argmax(model(x))

### 위치보정 사용

안타깝지만 생각보다 결과가 잘 나오지 안을 것이다. 사실 우리가 사용하는 신경망은 어디까지나 픽셀의 고정위치의 밝기를 기준으로 하기 때문에 이미지의 크기와 여백등에 영향을 크게 받는다. 다음 함수 reloacte 는 numpy 이미지의 위치를 재 배치하고 일정 여백을 줘서 보정한다. 

In [ ]:
def relocate(src):  ## 숫자 이미지의 위치를 보정하고 여백을 추가한다.
  xsum = np.sum(src , axis = 0) > 0 
  ysum = np.sum(src , axis = 1) > 0 

  img = src[:, xsum]
  img = img[ysum, :]

  paddx = int( abs(img.shape[0] - img.shape[1] ) / 2 )
  return np.pad(img, ((10,10), (paddx+10,paddx+10)) )



이제 위의 함수를 사용해 이미지를 보정해보도록 하겠다. 

In [ ]:
img = Image.open("3.png").convert('L')
w,h = img.size
src = np.array( img.getdata() ).reshape(h,w)

src = 255 - src
src = relocate(src)
plt.imshow( src, cmap = 'gray' )
plt.show() 


또 다른 문제는 위의 이미지를 28x28 로 변환해야한다는 것인데.. 이것은 nn.functional 의 interpolate 함수를 사용한다. 다만 이 함수는 앞에 2개의 더미차원을 요구하기 때문에 다음과 같이 실행한다. 

In [ ]:
import torch.nn.functional as F 

x = tc.FloatTensor(src)
x = F.interpolate( x[None,None,:,:], (28,28) )

plt.imshow(x.view(28,28), cmap = 'gray')

이제 준비가 되었으니 위의 x 를 reshape 해서 cuda 에 올려 결과를 판별해보도록 하겠다. 

In [ ]:
x = x.to(cuda).reshape(28*28) 
print( tc.argmax( model( x ) ) )


### 여러 숫자 이미지 판별

끝으로  다음과 같은 여러 숫자가 들어있는 이미지를 이용해 판별하는 코드를 한번 구경하자
<img src = "https://github.com/decoz/mlclass/blob/master/images/t_nums2.png?raw=true" align = "left">


In [ ]:
im = Image.open("t_nums2.png").convert('L')
w,h = im.size
im = np.array(im.getdata()).reshape(h,w)
plt.imshow(im, cmap = 'gray')

다음은 이런 이미지의 빈 영역을 삭제하고 이미지를 잘라낸 후 이미지를 보정하는  코드이다. 

In [ ]:
ysum = np.sum(im, axis = 0)

thresh = 0
s = -1

for x in range(w):
  if s >= 0 and ysum[x] <= thresh : 
    src = relocate( im[:, s:x] )
    plt.imshow(src)
    plt.show()
 
    s = -1 

  elif s < 0 and ysum[x] > thresh  :
    s = x




### <font color = '#FF7050'> 실습 : 여러 숫자값 판별하기 
위의 코드를 이용해 여러 숫자가 들어있는 이미지를 판별하도록 해보자
</font>

In [ ]:
## 이 코드를 수정해 여러 숫자가 들어있는 이미지를 판별할 수 있도록 해보세요
ysum = np.sum(im, axis = 0)

thresh = 0
s = -1

for x in range(w):
  if s >= 0 and ysum[x] <= thresh : 
    src = relocate( im[:, s:x] )
    plt.imshow(src)
    plt.show()
 
    s = -1 

  elif s < 0 and ysum[x] > thresh  :
    s = x
